In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM
import evaluate
import numpy as np

In [5]:
import wandb


wandb.init(
    project="briefly-v1",  
    name="briefly-trialsv1",    
    config={                   
        "learning_rate": 2e-5,
        "epochs": 4,
        "batch_size": 16
    }
)

In [7]:
billsum = load_dataset("billsum", split="ca_test")

In [8]:
billsum = billsum.train_test_split(test_size = 0.25)

In [9]:
billsum['train'][2]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 51203 of the Government Code is amended to read:\n51203.\n(a) The assessor shall determine the current fair market value of the land as if it were free of the contractual restriction pursuant to Section 51283. The Department of Conservation or the landowner, also referred to in this section as “parties,” may provide information to assist the assessor to determine the value. Any information provided to the assessor shall be served on the other party, unless the information was provided at the request of the assessor, and would be confidential under law if required of an assessee.\n(b) Within 45 days of receiving the assessor’s notice pursuant to subdivision (a) of Section 51283 or Section 51283.4, if the Department of Conservation or the landowner believes that the current fair market valuation certified pursuant to subdivision (b) of Section 51283 or Section 51283.4 is not accurate, the depart

In [10]:
model_checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
prefix_prompt = "Summarize: "

In [12]:
def preprocess_text(txt):
    inputs = [prefix_prompt + doc for doc in txt["text"]]
    tokenix = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=txt["summary"], max_length=128, truncation=True)
    tokenix["labels"] = labels["input_ids"]
    return tokenix

In [13]:
billsum_tokeniz = billsum.map(preprocess_text, batched=True)

Map: 100%|██████████| 310/310 [00:00<00:00, 970.33 examples/s] 


In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

In [15]:
rouge = evaluate.load("rouge")
rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [16]:
def compute_metrics(pred):
    preds, labels = pred
    decode_pred = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_type_id)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    rouge_metric = rouge.compute(predictions=decode_pred, references=decode_labels, use_stemmer = True)
    prediction_lens = [np.count_nonzero(pred!=tokenizer.pad_token_type_id) for pred in preds]
    rouge_metric["gen_len"] = np.mean(prediction_lens)
    return {k: round(v,4) for k,v in rouge_metric.items()}

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="text_summarizerv1",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16, 
    weight_decay=0.01,
    save_total_limit=3, 
    num_train_epochs=4, 
    predict_with_generate=True,
    report_to="wandb",
    fp16=True, 
)

In [19]:
trainer =Seq2SeqTrainer(
    model = model, 
    args= training_args,
    train_dataset=billsum_tokeniz["train"], 
    eval_dataset=billsum_tokeniz["test"],
    processing_class=tokenizer, 
    data_collator= data_collator, 
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
 95%|█████████▌| 19/20 [00:13<00:00,  1.25it/s]
                                                


{'eval_loss': 2.8220036029815674, 'eval_rouge1': 0.1283, 'eval_rouge2': 0.0393, 'eval_rougeL': 0.1055, 'eval_rougeLsum': 0.1055, 'eval_gen_len': 20.0, 'eval_runtime': 16.0072, 'eval_samples_per_second': 19.366, 'eval_steps_per_second': 1.249, 'epoch': 1.0}


 95%|█████████▌| 19/20 [00:12<00:00,  1.39it/s]
                                                 


{'eval_loss': 2.5943057537078857, 'eval_rouge1': 0.1374, 'eval_rouge2': 0.0482, 'eval_rougeL': 0.1121, 'eval_rougeLsum': 0.1121, 'eval_gen_len': 20.0, 'eval_runtime': 14.8041, 'eval_samples_per_second': 20.94, 'eval_steps_per_second': 1.351, 'epoch': 2.0}


 95%|█████████▌| 19/20 [00:12<00:00,  1.40it/s]
                                                 


...

{'eval_loss': 2.5171501636505127, 'eval_rouge1': 0.1434, 'eval_rouge2': 0.0516, 'eval_rougeL': 0.118, 'eval_rougeLsum': 0.1178, 'eval_gen_len': 20.0, 'eval_runtime': 14.8075, 'eval_samples_per_second': 20.935, 'eval_steps_per_second': 1.351, 'epoch': 4.0}


100%|██████████| 20/20 [00:14<00:00,  1.64it/s]


{'train_runtime': 269.4984, 'train_samples_per_second': 13.759, 'train_steps_per_second': 0.861, 'train_loss': 3.109038780475485, 'epoch': 4.0}


100%|██████████| 232/232 [04:29<00:00,  1.16s/it]


TrainOutput(global_step=232, training_loss=3.109038780475485, metrics={'train_runtime': 269.4984, 'train_samples_per_second': 13.759, 'train_steps_per_second': 0.861, 'total_flos': 1003694799716352.0, 'train_loss': 3.109038780475485, 'epoch': 4.0})

In [21]:
wandb.finish()

eval/gen_len,▁▁▁▁
eval/loss,█▃▁▁
eval/rouge1,▁▅▇█
eval/rouge2,▁▆▇█
eval/rougeL,▁▅▇█
eval/rougeLsum,▁▅▇█
eval/runtime,█▁▁▁
eval/samples_per_second,▁███
eval/steps_per_second,▁███
train/epoch,▁▃▆██
train/global_step,▁▃▆██


In [22]:
model.save_pretrained('textsum-v1-model')
tokenizer.save_pretrained('tokenix-textsum-v1')

('tokenix-textsum-v1\\tokenizer_config.json',
 'tokenix-textsum-v1\\special_tokens_map.json',
 'tokenix-textsum-v1\\spiece.model',
 'tokenix-textsum-v1\\added_tokens.json',
 'tokenix-textsum-v1\\tokenizer.json')